Installing or upgrading the bitsandbytes library, which is used to perform model quantization, reducing model size without significant accuracy loss.

In [ ]:
pip install --upgrade bitsandbytes

This code imports the login function from the huggingface_hub library and initiates the login process to Hugging Face Hub. The login function allows authentication using a personal access token, which you replace with your own token in the token parameter.

In [ ]:
from huggingface_hub import login

login(
  token="*********", # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


This code imports the necessary libraries and configurations to load a quantized Hugging Face model. First, it specifies the model ID for Meta's Llama-3.1-8B-Instruct model.

Using BitsAndBytesConfig, it configures the model to load in 4-bit precision (load_in_4bit=True), with double quantization (bnb_4bit_use_double_quant=True) and "nf4" quantization type (bnb_4bit_quant_type="nf4") to save memory while preserving accuracy.

The configuration also sets computation to use float16 for efficient processing on GPU. The model is then loaded using AutoModelForCausalLM.from_pretrained with automatic device mapping and quantization settings, and its corresponding tokenizer is loaded and set up to use the eos_token as its padding token.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face model id
model_id = "meta-llama/Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16  # Use fp16 instead of bf16
)


# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Performs a GPU utilization check after model quantization using the !nvidia-smi command. This command helps monitor GPU resources, including memory usage and temperature, to ensure the quantized model is running efficiently and there is adequate GPU memory available.

In [ ]:
!nvidia-smi

Sun Nov  3 14:03:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0              27W /  70W |   5647MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
prompt = "Explain what is machine learning in simple terms"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

In [ ]:
outputs = model.generate(
    inputs.input_ids,
    max_length=512,
    temperature=0.3,
    do_sample=True
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
from pprint import pprint

pprint(response)

('Explain what is machine learning in simple terms\n'
 'Machine learning is a subset of artificial intelligence (AI) that involves '
 'training algorithms to make predictions or decisions based on data. In '
 'simple terms, machine learning is like teaching a computer to learn from '
 'experience, just like how humans do.\n'
 "Imagine you're trying to teach a child to recognize different animals. You "
 'show them pictures of cats, dogs, and birds, and say "this is a cat," "this '
 'is a dog," and "this is a bird." Over time, the child learns to recognize '
 'the characteristics of each animal and can make predictions about what kind '
 "of animal a new picture is. That's basically what machine learning does, but "
 'with computers and data instead of children and pictures.\n'
 'There are three main types of machine learning:\n'
 '1. **Supervised learning**: The computer is shown examples of data and their '
 'corresponding labels (like "this is a cat"). The computer learns to make '
 